# 🔌 VSCode Remote Tunnel - Debug GPU sur Colab

Ce notebook établit une connexion directe entre votre VSCode local et Colab.

**Avantages**:
- ✅ Breakpoints dans VSCode
- ✅ Step trace (F10/F11)
- ✅ Variables inspection
- ✅ Terminal interactif GPU
- ✅ Édition directe des fichiers Colab
- ✅ Copilot avec contexte GPU

**Durée setup**: ~2 minutes

**Prérequis**: Extension VSCode "Remote - Tunnels" (`ms-vscode.remote-server`)

## 📋 Étape 1: Vérifier GPU

In [ ]:
# Vérifier que GPU est activé
!nvidia-smi --query-gpu=name,memory.total,driver_version --format=csv

import torch
if torch.cuda.is_available():
    print(f"\n✅ GPU détecté: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("\n⚠️ GPU non détecté - Vérifiez Runtime → Change runtime type → GPU")

## 📥 Étape 2: Télécharger VSCode CLI

In [ ]:
%%bash
# Télécharger VSCode CLI (Alpine Linux version pour Colab)
if [ ! -f "code" ]; then
    echo "📥 Téléchargement VSCode CLI..."
    curl -Lk 'https://code.visualstudio.com/sha/download?build=stable&os=cli-alpine-x64' \
      --output vscode_cli.tar.gz
    
    echo "📦 Extraction..."
    tar -xf vscode_cli.tar.gz
    
    echo "✅ VSCode CLI installé"
    ./code --version
else
    echo "✅ VSCode CLI déjà installé"
    ./code --version
fi

## 🔑 Étape 3: Authentification GitHub

**IMPORTANT**: Cette cellule va afficher un code à 8 caractères.

Suivez les instructions:
1. Copiez le code affiché (ex: `ABCD-1234`)
2. Ouvrez https://github.com/login/device dans un nouvel onglet
3. Collez le code
4. Cliquez "Continue" puis "Authorize"
5. Revenez ici - l'authentification sera complète

In [ ]:
# Authentification GitHub (device flow - pas de callback)
!./code tunnel user login --provider github

## 🚀 Étape 4: Lancer le Tunnel

Cette cellule va créer le tunnel VSCode.

**URLs affichées**:
- `vscode.dev/tunnel/...` = VSCode Web (navigateur)
- `vscode://vscode-remote/tunnel/...` = VSCode Desktop (recommandé)

**Note**: Cette cellule tourne en continu (normal). Ne pas l'interrompre!

In [ ]:
# Lancer tunnel (reste actif)
!./code tunnel --accept-server-license-terms --name colab-panini-debug

## 💻 Étape 5: Connecter VSCode Local

**Dans votre VSCode local**:

1. Installer extension si pas déjà fait:
   - `Ctrl+Shift+X` → Chercher "Remote - Tunnels"
   - Installer `ms-vscode.remote-server`

2. Connecter au tunnel:
   - `Ctrl+Shift+P` → "Remote-Tunnels: Connect to Tunnel"
   - Login avec votre compte GitHub
   - Sélectionner `colab-panini-debug` dans la liste

3. **Nouvelle fenêtre VSCode s'ouvre** connectée à Colab! 🎉

**Alternative rapide**: Cliquer sur le lien `vscode://...` affiché ci-dessus

## 📁 Étape 6: Setup Workspace (Exécuter dans NOUVELLE fenêtre VSCode)

Une fois connecté au tunnel, exécutez ces commandes dans le **terminal de la nouvelle fenêtre VSCode** (pas ici!):

```bash
# 1. Clone repo
git clone https://github.com/stephanedenis/Panini.git /content/work
cd /content/work

# 2. Install dependencies
pip install -r requirements.txt -q

# 3. Mount Google Drive (optionnel - persistance)
# from google.colab import drive
# drive.mount('/content/drive')

# 4. Vérifier GPU
python -c "import torch; print(f'GPU: {torch.cuda.get_device_name(0)}')"
```

Ou copiez-collez les cellules ci-dessous dans le terminal VSCode.

In [ ]:
# Alternative: Exécuter setup depuis ce notebook
import os
from pathlib import Path

# Clone repo si pas déjà fait
if not Path("/content/work").exists():
    print("📥 Clonage du repo...")
    !git clone https://github.com/stephanedenis/Panini.git /content/work
else:
    print("✅ Repo déjà cloné")
    os.chdir("/content/work")
    !git pull origin gpu-experiments

os.chdir("/content/work")
print(f"\n📂 Working directory: {os.getcwd()}")

# Install dependencies
print("\n📦 Installation des dépendances...")
!pip install -r requirements.txt -q

print("\n✅ Setup terminé - Workspace prêt pour debugging!")

## 🎯 Étape 7: Commencer à Debugger!

**Dans la fenêtre VSCode connectée au tunnel**:

1. **Ouvrir fichier**: `experiments/benchmark_audio_fingerprinting.py`

2. **Mettre breakpoint**: Cliquer à gauche d'une ligne (point rouge)

3. **Créer configuration debug**: Créer `.vscode/launch.json`:
   ```json
   {
     "version": "0.2.0",
     "configurations": [
       {
         "name": "Debug Audio Fingerprinting",
         "type": "python",
         "request": "launch",
         "program": "${file}",
         "console": "integratedTerminal",
         "args": ["--num-samples", "10"],
         "env": {
           "CUDA_VISIBLE_DEVICES": "0"
         }
       }
     ]
   }
   ```

4. **Lancer debug**: `F5` ou Run → Start Debugging

5. **Contrôles debug**:
   - `F5` = Continue
   - `F10` = Step Over (ligne suivante)
   - `F11` = Step Into (entrer dans fonction)
   - `Shift+F11` = Step Out (sortir de fonction)
   - `Shift+F5` = Stop

6. **Panels utiles**:
   - **Variables**: Voir toutes les variables locales
   - **Watch**: Expressions à surveiller
   - **Call Stack**: Pile d'appels
   - **Debug Console**: Évaluer expressions en live

**Exemple de debug**:
```python
# Dans experiments/debug_example.py
import torch

def test_gpu():
    device = torch.device("cuda")  # Breakpoint ici
    x = torch.randn(1000, 1000).to(device)  # F10 pour arriver ici
    y = x @ x.T  # F10 encore
    print(f"Result: {y.shape}")  # Hover sur y pour voir valeur

test_gpu()
```

## 📊 Monitoring GPU (optionnel)

In [ ]:
# Voir utilisation GPU en temps réel
!nvidia-smi --query-gpu=timestamp,name,utilization.gpu,memory.used,memory.total --format=csv -l 2

## 🔄 Reconnexion Rapide

Si le tunnel se déconnecte (déconnexion réseau, etc.):

1. Dans VSCode: `Ctrl+Shift+P` → "Remote-Tunnels: Connect to Tunnel"
2. Sélectionner `colab-panini-debug`
3. ✅ Reconnecté!

Si la VM Colab redémarre:
1. Re-exécuter toutes les cellules de ce notebook
2. Reconnexion VSCode (étapes ci-dessus)

## 💾 Persistance avec Google Drive

In [ ]:
# Monter Google Drive pour sauvegarder outputs
from google.colab import drive
drive.mount('/content/drive')

# Créer symlink vers outputs
import os
os.makedirs('/content/drive/MyDrive/colab_debug_outputs', exist_ok=True)
!ln -sf /content/drive/MyDrive/colab_debug_outputs /content/work/outputs

print("✅ Outputs seront sauvegardés dans Google Drive")

## 🎓 Tips & Tricks

### Debug Performance GPU
```python
# Profiler PyTorch avec breakpoint
import torch
from torch.profiler import profile, ProfilerActivity

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
    # Votre code GPU ici
    x = torch.randn(1000, 1000, device='cuda')
    y = x @ x.T  # Breakpoint ici

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))
```

### Conditional Breakpoints
```python
# Dans VSCode, right-click sur breakpoint → Edit Breakpoint
# Expression: i == 50 (s'arrête seulement si i vaut 50)
for i in range(100):
    process(i)  # Breakpoint conditionnel
```

### Watch Expressions
```python
# Dans panel Watch, ajouter:
# - torch.cuda.memory_allocated() / 1e9
# - x.shape
# - x.device
```

### Debug Console
```python
# Pendant pause sur breakpoint, dans Debug Console:
>>> x.mean().item()
>>> torch.cuda.get_device_name(0)
>>> dir(model)
```

## 🆘 Troubleshooting

### "Tunnel not found"
**Fix**: Re-exécuter cellules 3 et 4 (auth + tunnel)

### "Extension not installed"
**Fix**: Dans VSCode local, installer "Remote - Tunnels" (`ms-vscode.remote-server`)

### "Import error" dans debugging
**Fix**: Re-exécuter cellule 6 (install dependencies)

### "GPU not available" dans VSCode
**Fix**: 
1. Vérifier Runtime → Change runtime type → GPU
2. Re-exécuter cellule 1 (GPU check)

### Tunnel très lent
**Cause**: Latence réseau normale
**Alternative**: Utiliser Solution 2 (daemon) pour batch jobs